--- Day 2: Cube Conundrum ---

You're launched high into the atmosphere! The apex of your trajectory just barely reaches the surface of a large island floating in the sky. You gently land in a fluffy pile of leaves. It's quite cold, but you don't see much snow. An Elf runs over to greet you.

The Elf explains that you've arrived at Snow Island and apologizes for the lack of snow. He'll be happy to explain the situation, but it's a bit of a walk, so you have some time. They don't get many visitors up here; would you like to play a game in the meantime?

As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.

To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.

You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).

In [57]:
import pandas as pd
import numpy as np

In [58]:
with open('2023_day2.csv', 'r') as f:
    s = f.readlines()
    df = pd.DataFrame(s)
df.head()

,0
0,"Game 1: 1 green, 2 blue; 13 red, 2 blue, 3 gre..."
1,"Game 2: 2 blue, 11 green; 4 blue, 12 red, 4 gr..."
2,"Game 3: 1 blue, 12 green, 2 red; 9 red, 16 gre..."
3,"Game 4: 8 green, 18 blue; 4 green, 14 blue, 2 ..."
4,"Game 5: 7 red, 15 blue, 1 green; 13 blue; 18 r..."


In [59]:
r = df[0].str.split(':', expand=True).rename(columns={0:'id'})
r['id'] = r['id'].str.replace('Game ', '').astype(int)
r[1] = r[1].str.replace('\n','')
r = pd.concat([r['id'], r[1].str.split(';', expand=True)], axis=1)
r.head()

,id,0,1,2,3,4,5
0,1,"1 green, 2 blue","13 red, 2 blue, 3 green","4 green, 14 red",None,None,None
1,2,"2 blue, 11 green","4 blue, 12 red, 4 green","7 red, 1 blue, 9 green","10 green, 12 red, 6 blue",None,None
2,3,"1 blue, 12 green, 2 red","9 red, 16 green","1 red, 10 green, 1 blue","1 red, 14 green",None,None
3,4,"8 green, 18 blue","4 green, 14 blue, 2 red","3 blue, 5 green, 11 red",None,None,None
4,5,"7 red, 15 blue, 1 green",13 blue,"18 red, 2 green, 9 blue","19 blue, 5 green, 10 red","9 green, 2 blue, 7 red",None


In [60]:
col_count = len([x for x in r.columns if x != 'id'])
print(col_count)

6


In [61]:
def find_max(df, color):
    m = 0
    for c in range(col_count):
        if df[c] is not None:
            v = 0
            sets = df[c].split(',')
            for s in sets:
                if s.find(color) > 0:
                    v = int(s.strip().split(' ')[0])
            if v > m:
                m = v
    return m

In [62]:
r['maxred'] = r.apply(lambda x: find_max(x, 'red'), axis=1)
r['maxblue'] = r.apply(lambda x: find_max(x, 'blue'), axis=1)
r['maxgreen'] = r.apply(lambda x: find_max(x, 'green'), axis=1)
r.head()

,id,0,1,2,3,4,5,maxred,maxblue,maxgreen
0,1,"1 green, 2 blue","13 red, 2 blue, 3 green","4 green, 14 red",None,None,None,14,2,4
1,2,"2 blue, 11 green","4 blue, 12 red, 4 green","7 red, 1 blue, 9 green","10 green, 12 red, 6 blue",None,None,12,6,11
2,3,"1 blue, 12 green, 2 red","9 red, 16 green","1 red, 10 green, 1 blue","1 red, 14 green",None,None,9,1,16
3,4,"8 green, 18 blue","4 green, 14 blue, 2 red","3 blue, 5 green, 11 red",None,None,None,11,18,8
4,5,"7 red, 15 blue, 1 green",13 blue,"18 red, 2 green, 9 blue","19 blue, 5 green, 10 red","9 green, 2 blue, 7 red",None,18,19,9


The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?

In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get 8.

In [63]:
r['possible'] = np.where((r['maxred'] <= 12) & (r['maxblue'] <= 14) & (r['maxgreen'] <= 13), 1, 0)
print(f"Total Possible: {r[r['possible'] == 1]['id'].sum()}")
r.head()

Total Possible: 2541


,id,0,1,2,3,4,5,maxred,maxblue,maxgreen,possible
0,1,"1 green, 2 blue","13 red, 2 blue, 3 green","4 green, 14 red",None,None,None,14,2,4,0
1,2,"2 blue, 11 green","4 blue, 12 red, 4 green","7 red, 1 blue, 9 green","10 green, 12 red, 6 blue",None,None,12,6,11,1
2,3,"1 blue, 12 green, 2 red","9 red, 16 green","1 red, 10 green, 1 blue","1 red, 14 green",None,None,9,1,16,0
3,4,"8 green, 18 blue","4 green, 14 blue, 2 red","3 blue, 5 green, 11 red",None,None,None,11,18,8,0
4,5,"7 red, 15 blue, 1 green",13 blue,"18 red, 2 green, 9 blue","19 blue, 5 green, 10 red","9 green, 2 blue, 7 red",None,18,19,9,0
